This pipeline extracts annotated entities and labels from training data for every annotator and preprocesses data for calculating inter-annotator agreement and calculates entity composition for each file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Download packages
!python3 -m spacy download en
!python3 -m spacy download en_core_web_sm

#Import libraries
import json 
import os
import glob
import spacy
import random
import en_core_web_sm
from spacy import displacy
from pathlib import Path

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 12.0 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


Compute length and number of words for every file in training, testing and gold datasets

In [ ]:
#Define the source folder
import collections
training_txt_path='/content/drive/MyDrive/Bushire-GT-compound/training_data_txt/'
testing_txt_path='/content/drive/MyDrive/Bushire-GT-compound/inputs/'
gold_txt_path='/content/drive/MyDrive/Bushire-GT-compound/gold_data/gold_inputs/'

def compute_lengths(path,input_lengths):
  new_path=path+"*.txt"
  #Load all json files from the source folder and append them to the dictionary 
  for f in glob.glob(new_path):
        try:
          infile=open(f,encoding='utf8')
          #extract name of the file
          name=infile.name.split("/")[-1]
          print(name)
          index=name.find(".txt")
          name=name[:index]
          #read file
          text=infile.read()
        except UnicodeDecodeError:
          infile=open(f,encoding='windows-1252')
          #extract name of the file
          name=infile.name.split("/")[-1]
          index=name.find(".txt")
          name=name[:index]
          #read file
          text=infile.read()
        text=text.replace("\n", " ")
        text=text.replace("\'", " ")
        words = text.split()
        input_lengths[name]=(len(text),len(words))
 
#the dictionary consists of a tuple where first value = text length and second value=number of words
gold_txt_lengths=collections.defaultdict(int)
compute_lengths(gold_txt_path,gold_txt_lengths)

training_txt_lengths=collections.defaultdict(int)
compute_lengths(training_txt_path,training_txt_lengths)

testing_txt_lengths=collections.defaultdict(int)
compute_lengths(testing_txt_path,testing_txt_lengths)


xby.txt
xin.txt
1825OCR_page15.txt
iraq3.txt
iraq1.txt
iraq2.txt
iraq4.txt
iraq5.txt
xaa.txt
xah.txt
xai.txt
xcd.txt
xcm.txt
xeq.txt
xla.txt
xad.txt
xac.txt
xae.txt
xaf.txt
xbi.txt
xbj.txt
xab.txt
xaa.txt
iraq.txt
xad.txt
Book271,_pp_1-100.txt
1825OCR_page23.txt
1825OCR_page15.txt
1825OCR_page12.txt


In [ ]:
training_txt_lengths

defaultdict(int,
            {'iraq1': (3937, 646),
             'iraq2': (3812, 651),
             'iraq4': (4282, 705),
             'iraq5': (4220, 737),
             'xaa': (3505, 623),
             'xad': (2628, 470),
             'xah': (2351, 419),
             'xai': (2530, 448),
             'xcd': (3293, 609),
             'xcm': (3859, 659),
             'xeq': (1746, 316),
             'xla': (2004, 346)})

In [ ]:
testing_txt_lengths

defaultdict(int,
            {'1825OCR_page12': (1171, 209),
             '1825OCR_page15': (1424, 253),
             '1825OCR_page23': (1594, 273),
             'Book271,_pp_1-100': (69749, 12123),
             'iraq': (272796, 46474),
             'xaa': (2907, 515),
             'xab': (1803, 315),
             'xac': (3542, 628),
             'xad': (2628, 470),
             'xae': (2347, 415),
             'xaf': (2664, 475),
             'xbi': (1791, 318),
             'xbj': (2187, 383)})

In [ ]:
gold_txt_lengths

defaultdict(int,
            {'1825OCR_page15': (1424, 253),
             'iraq3': (2517, 425),
             'xby': (2332, 427),
             'xin': (1458, 248)})

In [ ]:
def print_lengths(dict_path,string):
  total_length=0
  total_symbols=0
  for filename, tuple in dict_path.items():
    total_length+=tuple[1]
    total_symbols+=tuple[0]
  print(string+" data: number of words: "+str(total_length)+" and number of symbols: "+str(total_symbols))


print_lengths(gold_txt_lengths,"gold")
print_lengths(training_txt_lengths,"training")


gold data: number of words: 1353 and number of symbols: 7731
training data: number of words: 6629 and number of symbols: 38167


Load training data from every annotator and store them in a dictionary

In [ ]:
#Define the source folder
source_path='/content/drive/MyDrive/Bushire-GT-compound/gold_data/gold_inputs/'
#source_path='/content/drive/MyDrive/Bushire-GT-compound/training_data_json/'

#Load all json files from the source folder and append them to the dictionary 
train_files = {} #-- name stores file name and value stores full json content
train_annotations={} #-- name stores file name and value stores json content - only annotation section
annotated_r={}
annotated_d={}
annotated_s={}
for f in glob.glob(source_path+"*.json"):
    with open(f) as infile:
        name=infile.name.split("/")[-1]
        index=name.find(".json")
        name=name[:index]
        new_index=name.find("-")
        file_content=json.load(infile)
        if new_index!=-1:
          char=name[-1]
          if char=="r":
            annotated_r[name]=file_content['annotations']
          elif char=="s":
            annotated_s[name]=file_content['annotations']
          elif char=="d":
            annotated_d[name]=file_content['annotations']
        train_files[name]=file_content['annotations']
        #Store annotation section of each json file in a separate array
        #if 'annotations' in train_files[name]:
        train_annotations[name]=file_content['annotations']

print(annotated_d)


{'xby-d': [['Mirza Mehedy Ally Khawn', {'entities': [[0, 23, 'PERSON']]}], ['1802', {'entities': [[0, 4, 'DATE']]}], ['Resident', {'entities': [[0, 8, 'TITLE']]}], ['Wm Bruce Asst.', {'entities': [[0, 13, 'PERSON']]}], ['Mehedi Alli Khawn', {'entities': [[0, 17, 'PERSON']]}], ['Resident at Bushier', {'entities': [[0, 8, 'TITLE'], [12, 19, 'GPE']]}], ['In the 2nd. Instant I received your letter of the 27th. Ultimo- The alert now returns', {'entities': [[7, 19, 'DATE'], [50, 63, 'DATE']]}], ['to Bushire on her way to the Presidency and I request that you will expedite her departure from', {'entities': [[3, 10, 'GPE'], [29, 39, 'ORG']]}], ['Bushire as much as may be possible', {'entities': [[0, 7, 'GPE']]}], ['remitting Me per Alert the Sum of Piasters 24000 / on the account of the Honble Company and', {'entities': [[34, 48, 'MONEY'], [73, 87, 'ORG']]}], ['I am to request that you will not in the present State of the Bushire Treasury think', {'entities': [[62, 78, 'ORG']]}], ['The Invoice

In [ ]:
annotated_r

Extract entity and label (as a tuple) for every text file from every annotator and calculated Named Entity composition for every file

In [ ]:
from collections import defaultdict
total_labels=defaultdict(int)
total_text_labels={}
def populate(annotated_diction,total_labels,total_text_labels):
  all_files_entities=defaultdict(list)
  for filename in annotated_diction.keys():
    orig_filename=filename
    content=annotated_diction[filename]
    index=filename.find("-")
    filename=filename[:index]
    all_entities=[]
    for content_file in content:
      annotation=content_file[0]
      #annotation=annotation.replace("\'", "")
      #annotation=annotation.replace("'I", "I")
      entities=content_file[1]['entities']
      if entities:
        for ent in entities:
          start=ent[0]
          end=ent[1]
          label=ent[2]
          #increment number of labels
          total_labels[label]+=1
          #increment number of labels per textfile
          if orig_filename not in total_text_labels:
            total_text_labels[orig_filename]={}
          if label in total_text_labels[orig_filename]:
            total_text_labels[orig_filename][label]+=1
          else:
            total_text_labels[orig_filename][label]=1
          entity_content=annotation[start:end]
          tuple=(entity_content,label)
          all_entities.append(tuple)
    if all_entities:
      all_files_entities[filename].append(all_entities)
  return all_files_entities

r_entities=populate(annotated_r,total_labels,total_text_labels)
s_entities=populate(annotated_s,total_labels,total_text_labels)
d_entities=populate(annotated_d,total_labels,total_text_labels)


In [ ]:
s_entities['xaa']

[]

In [ ]:
total_label_count=0
for label,count in total_labels.items():
  total_label_count+=count
  print(label+" "+str(count))
print(total_label_count)
print(len(total_labels))

PERSON 22
DATE 29
TITLE 28
GPE 33
ORG 23
VESSEL 15
MONEY 4
COMMODITY 5
GPE_ORG 1
STRUCTURE 6
QUANTITY 5
TRIBE 2
LOC 1
174
13


In [ ]:
total_label_count=collections.defaultdict(int)
for filename in total_text_labels:
  labels=total_text_labels[filename]
  total=0
  for key in labels:
    total+=labels[key]
  total_label_count[filename]+=total
print(total_label_count)
  


defaultdict(<class 'int'>, {'xby-s': 62, 'xby-d': 54, 'xin-d': 37, 'iraq3-d': 21})


Write every annotated file to the drive 

In [ ]:
#Format: every line contains a Named Entity and label
path="/content/drive/MyDrive/Bushire-GT-compound/gold_data/gold_annotator_output/"
def write_to_file(entity_dict,path,ext):
  for filename in entity_dict:
    out=entity_dict[filename][0]
    filepath=path+filename+ext+".txt"
    f = open((filepath), "w")
    for tuple in out:
      f.write(str(tuple[0])+" "+str(tuple[1]))
      f.write("\n")
    f.close()
write_to_file(r_entities,path,"-r")
write_to_file(d_entities,path,"-d")
write_to_file(s_entities,path,"-s")

Evaluate annotator's output and model output

In [ ]:
model_output_path="/content/drive/MyDrive/Bushire-GT-compound/gold_data/gold_outputs/"
annotator_output_path="/content/drive/MyDrive/Bushire-GT-compound/gold_data/gold_annotator_output/"
for f in glob.glob(annotator_output_path+"*.txt"):
      try:
        infile=open(f,encoding='utf8')
        #read file
        text=infile.read()
      except UnicodeDecodeError:
        infile=open(f,encoding='windows-1252')
        #read file
        text=infile.read()
      #extract name of the file
      name=infile.name.split("/")[-1]
      index=name.find(".txt")
      name=name[:index]
      new_index=name.find("-")
      orig_name=name[:new_index]
      text_arr=text.split("\n")
      for pair in text_arr:
        ent_arr=pair.split(" ")
        label=ent_arr[-1]
        entity=" ".join(ent_arr[0:len(ent_arr)-1])
        print(entity)


rupees
Honble Company
Rupees
Secretary
Honble Companys
Resident
Secretary-
Bushire
Bussora
Country Boat
Henry Moore
George Skipp
Mr. James Morley
Resident
Bushire
Express
Tyger
Coja
Stephan
2400KY
Dolphin Sohoonor
Carim Caun
Schiras
Sohooner
Honble Companys
Bushire
Residency-
Honble Company
Persian Dominions
Bushire
Bussora
Tyger
Rupees
three Thousand/3000/
Mr. Skipps
Schirass/
Bombay
Bushire
Mr Bouyears
Bushire

Government._
Bushire Resy
E.G. Stannus
3rd. Septr. 1826._
Rest._
Political Department
Secretary to Government
H.C.C. Clive
1st. Instant
1826
Political Department._
Bushire Resy
E.G. Stannus
3rd. Septr. 1826._
Rest._
Political Departt._
Secretary to Government
Lt. Coll. Macdonald
Chief Secretary
Bushire
3rd. Instant._
E.G. Stannus
Bushire Resy
Rest._
3rd. Septr. 1826._
Political Departt._
Secretary to Government
Hon'ble the Governor
His Royal Highness
Prince of Shiraz._
His Royal Highness
Zakee Khan
16th. August.
E.G. Stannus
Bushire Resy
Rest._
3rd. Septr. 1826._

'Irāq
Baghdā

In [ ]:
#Credits: https://towardsdatascience.com/inter-annotator-agreement-2f46c6d37bf3 
def cohen_kappa(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators.
    :param ann1: annotations provided by first annotator
    :type ann1: list
    :param ann2: annotations provided by second annotator
    :type ann2: list
    :rtype: float
    :return: Cohen kappa statistic
    """
    count = 0
    for an1, an2 in zip(ann1, ann2):
        if an1 == an2:
            count += 1
    A = count / len(ann1)  # observed agreement A (Po)

    uniq = set(ann1 + ann2)
    E = 0  # expected agreement E (Pe)
    for item in uniq:
        cnt1 = ann1.count(item)
        cnt2 = ann2.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E += count

    return round((A - E) / (1 - E), 4)



In [ ]:
import numpy as np
#Credits: https://www.datacamp.com/community/tutorials/fuzzy-string-python
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return ("The strings are {} edits away".format(distance[row][col]),distance[row][col])

Find approximately matching strings to solve the incorrect tokenization problem

In [ ]:
#we find at least 40% matching strings using the levenstein ratio formula
def matching_named_entities(annot1,annot2,matching,non_matching,annotations1,annotations2):
  for filename in annot1:
    if filename in annot2:
      #print(filename)
      content1=annot1[filename][0]
      content2=annot2[filename][0]
      #print(len(content1))
      for tuple1 in content1:        
        text1=tuple1[0]
        label1=tuple1[1]
        for tuple2 in content2:
          text2=tuple2[0]
          label2=tuple2[1]
          dist=levenshtein_ratio_and_distance(text1,text2)[1]
          len1=len(text1)
          len2=len(text2)
          if dist<(0.7*min(len1,len2)):
            annotations1.add(text1)
            annotations2.add(text2)
            if label1==label2:
              matching[label1].add((text1,text2))
            else:
              if (label1,label2) in non_matching:
                non_matching[(label1,label2)].add((text1,text2))
              elif (label2,label1) in non_matching:
                non_matching[(label2,label1)].add((text1,text2))
              else:
                non_matching[(label1,label2)].add((text1,text2))
          
annotations1=set()
annotations2=set()
matching=defaultdict(set)
non_matching=defaultdict(set)      
matching_named_entities(r_entities,s_entities,matching,non_matching,annotations1,annotations2)


In [ ]:
matching

defaultdict(set,
            {'GPE': {('Bahreen', 'hreen, '),
              ('Bombay', 'Bombay'),
              ('Bombay', 'mbay f'),
              ('Bombay', 'mbay; '),
              ('Bombay', 'ombay;'),
              ('Bushire', 'Bushire'),
              ('Bushire', 'Bushire-'),
              ('Bushire', 'Bushire-On'),
              ('Bushire', 'Bushire/'),
              ('Bushire', 'Bussora'),
              ('Bushire', 'Bussora/'),
              ('Bushire', 'shire b'),
              ('Bushire', 'shire i'),
              ('Bushire', 'shire s'),
              ('Bushire', 'shire, '),
              ('Bushire-', 'Bushire'),
              ('Bushire-', 'Bushire-'),
              ('Bushire-', 'Bushire-On'),
              ('Bushire-', 'Bushire/'),
              ('Bushire-', 'Bussora/'),
              ('Bushire-On', 'Bushire'),
              ('Bushire-On', 'Bushire-'),
              ('Bushire-On', 'Bushire-On'),
              ('Bushire-On', 'Bushire/'),
              ('Bushire/', 'Bushire'),

Evaluation

In [ ]:
train_annotations['xaa']

In [ ]:
r_entities['xaa']

[[('Mr. James Morley', 'PERSON '),
  ('Bushire', 'GPE'),
  ('Dolphin Schooner', 'VESSEL'),
  ('Carim Caun', 'PERSON '),
  ('Shaikhs of Boushire and Bah', 'PERSON '),
  ('Caun', 'TITLE'),
  ('Bushire', 'GPE'),
  ('Ormuso', 'GPE'),
  ('Bushire', 'GPE'),
  ('Shaik Ice', 'PERSON '),
  ('Bahreen', 'GPE'),
  ('Ormuso', 'GPE'),
  ('Dolphin', 'VESSEL'),
  ('Ormuse', 'GPE'),
  ('Mr Bonnyear', 'PERSON '),
  ('Bombay', 'GPE'),
  ('Gulph', 'LOC'),
  ('Salamander', 'VESSEL'),
  ('Bombay', 'GPE'),
  ('Lieutenant Gages', 'PERSON '),
  ('Bushire', 'GPE'),
  ('India', 'GPE'),
  ('Salamanders', 'VESSEL'),
  ('Bussora', 'GPE'),
  ('Mr Bowye', 'PERSON '),
  ('Bombay', 'GPE'),
  ('Mr Lyster', 'PERSON '),
  ('Bushire', 'GPE'),
  ('Mr. John Yoakly Botham', 'PERSON '),
  ('Wuter', 'PERSON '),
  ('Bushire', 'GPE'),
  ('Shaikhs', 'TITLE'),
  ('Schiras', 'GPE')]]